In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import vice
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from surp._globals import AGB_MODELS
import surp

In [ ]:
import arya

In [ ]:
tau_agb = 0.45
t_D = 0.0
gamma = 2
m_low = 1.0
m_high = 8

a_agb = surp.yield_models.C_AGB_Model(t_D=t_D, tau_agb=tau_agb, gamma=gamma, m_low=m_low)

In [ ]:
vice.mlr.larson1974(8)

In [ ]:
def recalc_dtd(tau_agb, t_D, gamma, m_low, times=np.logspace(-2.5, 1), m_high=8):
    t_high = vice.mlr.larson1974(m_low)
    t_low = vice.mlr.larson1974(m_high)

    tm = (times[1:] + times[:-1])/2
    R = (times - t_D)**gamma * np.exp(-(times-t_D) / tau_agb) * (times > t_D)
    R[times < t_low] = 0
    R[times > t_high] = 0
    y = np.cumsum(R[:-1]   * (times[1:] - times[:-1]))

    return tm, y / y[-1]

## Test analytic DTD

In [ ]:
def compare_act_exp(dt=0.01, **kwargs):
    a_agb = surp.yield_models.C_AGB_Model(**kwargs)
    
    vice.yields.agb.settings["c"] = a_agb
    vice.yields.ccsne.settings["c"] = 0
    vice.yields.sneia.settings["c"] = 0
    m_c, times = vice.single_stellar_population("c", Z=0.014 * 10**0, dt=dt)
    m_c = [c for c in m_c]
    
    plt.plot(times, np.array(m_c)/m_c[-1], lw=2, color="k")
    
    
    t, y = recalc_dtd(times=np.array(times), **kwargs)
    plt.plot(t, y, lw=2, ls="--", label="recalculated", color=arya.COLORS[1])
    plt.plot(t, y / np.array(m_c[1:]) * m_c[-1], ls=":", label="ratio")
    plt.xlabel("t/Gyr")
    plt.ylabel(r"$M(t)/M_{\rm tot}$")
    plt.xscale("log")
    plt.legend()
    plt.ylim(-0.3, 1.1)
    plt.xlim(0.03, 13.2)

In [ ]:
compare_act_exp(dt=0.0003, 
                tau_agb = 0.45,
    t_D = 0.1,
    gamma = 2,
    m_low = 0.08,
)

In [ ]:
compare_act_exp(dt=0.0001, 
                tau_agb = 0.45,
    t_D = 0.1,
    gamma = 2,
    m_low = 0.08,
)

In [ ]:
compare_act_exp(tau_agb = 0.45,
    t_D = 0.1,
    gamma = 0,
    m_low = 0.08,
)

In [ ]:
compare_act_exp(tau_agb = 0.45,
    t_D = 0.1,
    gamma = 1,
    m_low = 0.08,
)

In [ ]:
compare_act_exp(tau_agb = 1,
    t_D = 0.1,
    gamma = 1,
    m_low = 0.08,
)

In [ ]:
compare_act_exp(tau_agb = 1,
    t_D = 0.5,
    gamma = 1,
    m_low = 0.08,
)

In [ ]:
compare_act_exp(tau_agb = 1,
    t_D = 0.0,
    gamma = 1,
    m_low = 0.08,
)

In [ ]:
compare_act_exp(tau_agb = 2,
    t_D = 0.0,
    gamma = 1,
    m_low = 0.5,
)

## Comparisons

In [ ]:
tau_agb = 0.45
t_D = 0.0
gamma = 2
m_low = 1.0
m_high = 8

a_agb = surp.yield_models.C_AGB_Model(t_D=t_D, tau_agb=tau_agb, gamma=gamma, m_low=m_low)

In [ ]:

vice.yields.ccsne.settings["c"] = 0
vice.yields.sneia.settings["c"] = 0
vice.yields.ccsne.settings["fe"] = 0

for i in range(4):
    model = AGB_MODELS[i]
    vice.yields.agb.settings["c"] = model
    m_c, times = vice.single_stellar_population("c", Z=0.014 * 10**0)
    m_c = [c for c in m_c]
    plt.plot(times, np.array(m_c)/m_c[-1])


vice.yields.agb.settings["c"] = a_agb

m_c, times = vice.single_stellar_population("c", Z=0.014 * 10**0)
m_c = [c for c in m_c]

plt.plot(times, np.array(m_c)/m_c[-1], lw=2, color="k")



plt.xlabel("t/Gyr")
plt.ylabel(r"$M(t)/M_{\rm tot}$")
plt.xscale("log")
plt.legend()
plt.ylim(-0.3, 1.1)
plt.xlim(0.03, 13.2)

In [ ]:
for i in range(len(AGB_MODELS)):
    model = AGB_MODELS[i]

    vice.yields.agb.settings["c"] = model
    vice.yields.ccsne.settings["c"] = 0
    Zs = 0.014*10**np.linspace(-2, 1, 100)
    
    # plots importaint points
    if type(model) == str:
        y1, m1, z1 = vice.yields.agb.grid('c', study=model)
        Zs = np.array(z1)
    mass_yields = []
    for Z in Zs:
        m_c, times = vice.single_stellar_population("c", Z=Z)
        mass_yields.append(m_c[-1])
        
    y_c_agb = np.array(mass_yields)/1e6 
    plt.scatter(np.log10(Zs/0.014), y_c_agb)

   
# pass
""
vice.yields.agb.settings["c"] = a_agb

Zs = 0.014*10**np.linspace(-2, 0.5, 100)
mass_yields = []
for Z in Zs:
    m_c, times = vice.single_stellar_population("c", Z=Z)
    mass_yields.append(m_c[-1])

y_c_agb = np.array(mass_yields)/1e6 

plt.plot(np.log10(Zs/0.014), y_c_agb)
plt.xlabel("[Mg/H]")
plt.ylabel("$y_c$")